# 🤺 Fence demo

This notebook demonstrates the use of various classes in this package. 

In [38]:
%load_ext autoreload
from fence import Link, LLM, ClaudeInstantLLM, TransformationLink, Chain, LinearChain, PromptTemplate

## ⚙️ Setting up

In [39]:
# Get our model
claude = ClaudeInstantLLM(source='demo_notebook')

### 🚀 Level 1 - Just call the damn thing

In [40]:
# Use the invoke method to call the model
claude.invoke('Why is the sky blue?')

" The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue light there is, the more blue the sky appears."

In [41]:
# Just call the damn thing
claude('Why is the sky blue?')

" The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue light there is, the more blue the sky appears."

### 🚀🚀 Level 2 - Use a PromptTemplate

In [42]:
# Initialize a prompt template
prompt_template = PromptTemplate('Why is the sky {{color}}?', input_variables=['color'])
print(prompt_template)

PromptTemplate(source=Why is the sky {{color}}?, input_variables=['color'])


In [43]:
# Render it with a dictionary
print(prompt_template.render({'color': 'blue'}))

# Render it with keyword arguments
print(prompt_template.render(color='red'))

# Input dict takes precedence over keyword arguments
print(prompt_template.render(input_dict={'color': 'blue'}, color='red'))

Why is the sky blue?
Why is the sky red?
Why is the sky blue?


In [44]:
# You can concatenate prompt templates, input variables are merged
prompt_template_sky = PromptTemplate('Why is the sky {{color}}?', input_variables=['color'])
prompt_template_grass = PromptTemplate('Why is the grass {{color}}?', input_variables=['color'])
prompt_template_dress = PromptTemplate('I like a dress with {{pattern}}.', input_variables=['pattern'])
combined_prompt_template = prompt_template_sky + prompt_template_grass + prompt_template_dress
print(combined_prompt_template)
print(combined_prompt_template.render({'color': 'blue', 'pattern': 'polka dots'}))

PromptTemplate(source=Why is the sky {{color}}? Why is the grass {{color}}? I like a dress with {{pattern}}., input_variables=['pattern', 'color'])
Why is the sky blue? Why is the grass blue? I like a dress with polka dots.


In [45]:
# You can customize the separator
base_template = PromptTemplate('Why is the sky {{color}}?', input_variables=['color'], separator=' FUNKY TOWN ')
additional_template = PromptTemplate('Why is the grass {{color}}?', input_variables=['color'])
print((base_template + additional_template).render({'color': 'blue'}))

Why is the sky blue? FUNKY TOWN Why is the grass blue?


### 🚀🚀🚀 Level 3 - Use Links and Chains (lol)

What are Links? In this context, they represent atomic components of LLM interaction. That means they should be able to be strung together to form a Chain, although they can be used independently as well.

In [46]:
# The simplest link is the Link class, which just takes a prompt template and a model
link = Link(template=prompt_template, llm=claude)
print(link)

Link: <['color']> -> <state>


In [47]:
# Invoke it using the input_dict, NOT keyword arguments
link(input_dict={'color': 'blue'}) # Or, equivalently, link.run(input_dict={'color': 'blue'})

[14:52:31] INFO     🖇️ Executing  Link with input: {'color': 'blue'}                                                                                ]8;id=534213;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=764753;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\

[14:52:32] INFO     🧐 Current state:  The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in      ]8;id=662778;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=675421;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\
                    Earth's atmosphere, mostly nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the              
                    longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all                        
                    throughout the atmosphere and reach our eyes from every direction in the sky. The more scattered blue light from the sun                    
                    overwhelms the less scattered light of other colors, which is why we see the sky as blue.                                                   

{'state': " The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, mostly nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from every direction in the sky. The more scattered blue light from the sun overwhelms the less scattered light of other colors, which is why we see the sky as blue."}

In [48]:
# By default, output is stored under 'state'. You can get a copy (e.g., for inspection of intermediate results) by passing a different output key
link = Link(template=prompt_template, llm=claude, output_key='intermediate')
link.run(input_dict={'color': 'blue'})

           INFO     🖇️ Executing  Link with input: {'color': 'blue'}                                                                                ]8;id=252576;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=523795;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\

[14:52:34] INFO     🧐 Current state:  The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in      ]8;id=873933;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=936244;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\
                    Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than               
                    the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all                    
                    throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue light there is, the more blue              
                    the sky appears.                                                                                                                            

{'state': " The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue light there is, the more blue the sky appears.",
 'intermediate': " The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue light there is, the more b

In [49]:
# You can name your links for easier debugging in logs
link = Link(template=prompt_template, llm=claude, name='sky')
link.run(input_dict={'color': 'blue'})

           INFO     🖇️ Executing <sky> Link with input: {'color': 'blue'}                                                                           ]8;id=321475;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=927644;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\

[14:52:37] INFO     🧐 Current state:  The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in      ]8;id=517453;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=807045;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\
                    Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than               
                    the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all                    
                    throughout the atmosphere and reach our eyes from every direction in the sky. The more scattered blue light entering our eyes               
                    makes the sky appear blue.                                                                                                                  

{'state': " The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from every direction in the sky. The more scattered blue light entering our eyes makes the sky appear blue."}

In [50]:
# You can also build TransformationLinks, which take a function that transforms any input_dict into a specific output
def concatenate(x, y):
    return f"{x} and {y}"

concat_link = TransformationLink(
    input_keys=["X", "Y"], function=concatenate, output_key="C"
)

concat_link.run(input_dict={"X": "Hello", "Y": "World"})

{'state': 'Hello and World', 'C': 'Hello and World'}

In [51]:
# You can also build Chains, which are just a sequence of links. There are two types of chains: LinearChain and Chain. 
# LinearChain is a sequence of links, while Chain is a collection of links that are invoked in the right order based on the input and output keys for each Link.

# This is a LinearChain #
#########################

# Build some links
link_opposite = Link(
    template=PromptTemplate(
        "What's the opposite of {{A}}? Reply with a few words max.", ["A"]
    ),
    name = 'opposite',
    output_key="X",
)
link_superlative = Link(
    template=PromptTemplate(
        "What's a synonym for {{B}}. Reply with one word.", ["B"],
    ),
    name='superlative',
    output_key="Y",
)
link_poem = Link(
    template=PromptTemplate(
        "Write a poem about {{state}}. Return only the poem, beginning with the title.", ["state"]
    ),
    name='poem',
    output_key="Z",
)

# Now build a LinearChain
linear_chain = LinearChain(llm=claude, links=[link_opposite, link_superlative, concat_link, link_poem])

# Run it
result = linear_chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})

# Get the output
print(result['state'])

           INFO     🖇️ Executing <opposite> Link with input: {'A': 'A police officer', 'B': 'Hopeful'}                                              ]8;id=310414;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=810558;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\

[14:52:38] INFO     🧐 Current state:  Criminal                                                                                                    ]8;id=839734;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=35484;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\

           INFO     🔑 State keys: ['A', 'B', 'state', 'X'] (added: ['state', 'X'], removed: [])                                                   ]8;id=508754;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=915333;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#443\443]8;;\

           INFO     🖇️ Executing <superlative> Link with input: {'A': 'A police officer', 'B': 'Hopeful', 'state': ' Criminal', 'X': ' Criminal'}   ]8;id=311625;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=849253;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\

           INFO     🧐 Current state:  Optimistic                                                                                                  ]8;id=715855;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=41204;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\

...

           INFO     🔑 State keys: ['A', 'B', 'state', 'X', 'Y', 'C'] (added: ['C'], removed: [])                                                  ]8;id=93778;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=728400;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#443\443]8;;\

           INFO     🖇️ Executing <poem> Link with input: {'A': 'A police officer', 'B': 'Hopeful', 'state': ' Criminal and  Optimistic', 'X': '     ]8;id=385947;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=572219;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\
                    Criminal', 'Y': ' Optimistic', 'C': ' Criminal and  Optimistic'}                                                                            

[14:52:41] INFO     🧐 Current state:  The Criminal Optimist                                                                                       ]8;id=589777;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=767521;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\
                                                                                                                                                                
                    I've broken many laws without regret                                                                                                        
                    My actions others surely won't forget                                                                                                       
                    Yet still I dream that things will turn out right                                                                                           
                    Hoping the judge will go easy in her might                                                                                                  
                                                                                                                                                                
                    Behind these bars my mind begins to soar                                                                                                    
                    Imagining the life that's in store                                                                                                          
                    Once I've paid my debt to society                                                                                                           
                    The future's full of hope and possibility                                                                                                   
                                                                                                                                                                
                    My heart says this is just a minor slip                                                                                                     
                    That I'll learn from it, then get back on the trip                                                                                          
                    So I stay positive through each lonely night                                                                                                
                    Trusting better days will soon come into sight                                                                                              
                                                                                                                                                                
                    The criminal optimist, that's who I am                                                                                                      
                    Believing in redemption as best I can                                                                                                       
                    If I keep the faith that better's coming near                                                                                               
                    Then maybe hope is all a prisoner needs to hear                                                                                             

           INFO     🔑 State keys: ['A', 'B', 'state', 'X', 'Y', 'C', 'Z'] (added: ['Z'], removed: [])                                             ]8;id=543446;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=408910;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#443\443]8;;\

 The Criminal Optimist

I've broken many laws without regret  
My actions others surely won't forget
Yet still I dream that things will turn out right
Hoping the judge will go easy in her might  

Behind these bars my mind begins to soar  
Imagining the life that's in store  
Once I've paid my debt to society
The future's full of hope and possibility

My heart says this is just a minor slip  
That I'll learn from it, then get back on the trip
So I stay positive through each lonely night  
Trusting better days will soon come into sight

The criminal optimist, that's who I am  
Believing in redemption as best I can  
If I keep the faith that better's coming near  
Then maybe hope is all a prisoner needs to hear


In [52]:
# A LinearChain will take the presence of the 'state' key into account when invoking the next link.
# A Chain will not. However, it has an extra 'feature' in the form of topological sorting. As long as a graph of links can be
# extracted from the chain, and the input keys (that are not generated in the chain) are given, the chain will invoke the links in the right order.

# This is a Chain #
###################
link_a = Link(
    template=PromptTemplate(
        "Capitalize this word: {{A}}. Only respond with the capitalized version", ["A"]
    ),
    name = 'opposite',
    output_key="X",
)
link_b = Link(
    template=PromptTemplate(
        "What's a synonym for {{B}}. Reply with one word.", ["B"],
    ),
    name='superlative',
    output_key="Y",
)
link_c = Link(
    template=PromptTemplate(
        "Combine {{X}} and {{Y}} and {{C}} in a meaningful sentence.", ["X", "Y", "C"]
    ),
    name='sentence',
    output_key="Z",
)
chain = Chain(llm=claude, links=[link_c, link_a, link_b]) # Note that we can pass the links in any order

# This is the sorted graph of links
chain._topological_sort()

[Link: superlative <['B']> -> <Y>,
 Link: opposite <['A']> -> <X>,
 Link: sentence <['X', 'Y', 'C']> -> <Z>]

In [53]:
# Now we can run it
try:
    result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})
except Exception as e:
    print(e)

The following input keys are required: {'A', 'C', 'B'}. Missing: {'C'}


In [54]:
# Woops, forgot something! There's no link that generates the 'C' key. We can pass it in though.
result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful", "C": "a dog"})

           INFO     🖇️ Executing <superlative> Link with input: {'A': 'A police officer', 'B': 'Hopeful', 'C': 'a dog'}                             ]8;id=760752;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=904713;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\

[14:52:42] INFO     🧐 Current state:  Optimistic                                                                                                  ]8;id=951253;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=526451;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\

           INFO     🖇️ Executing <opposite> Link with input: {'A': 'A police officer', 'B': 'Hopeful', 'C': 'a dog', 'state': ' Optimistic', 'Y': ' ]8;id=372240;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=630778;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\
                    Optimistic'}                                                                                                                                

           INFO     🧐 Current state:  A Police officer.                                                                                           ]8;id=458972;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=893433;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\

           INFO     🖇️ Executing <sentence> Link with input: {'A': 'A police officer', 'B': 'Hopeful', 'C': 'a dog', 'state': ' A Police officer.', ]8;id=301208;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=490973;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#214\214]8;;\
                    'Y': ' Optimistic', 'X': ' A Police officer.'}                                                                                              

[14:52:43] INFO     🧐 Current state:  Here is a sentence combining those elements:                                                                ]8;id=901567;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py\chains.py]8;;\:]8;id=230759;file:///Users/wouter.durnez/Documents/Repositories/test-ai-message-composer/fence/src/llm/chains.py#270\270]8;;\
                                                                                                                                                                
                    The optimistic police officer's dog always believed in him and helped brighten his day, even after difficult calls.                         

In [55]:
# Cycles are not allowed
link_up = Link(
    template=PromptTemplate(
        "Capitalize this word: {{up}}. Only respond with the capitalized version", ["up"]
    ),
    name = 'up',
    output_key="down",
)
link_b = Link(
    template=PromptTemplate(
        "What's a synonym for {{down}}. Reply with one word.", ["down"],
    ),
    name='down',
    output_key="up",
)
chain = Chain(llm=claude, links=[link_up, link_b])
try:
    chain._topological_sort()
except Exception as e:
    print(e)

Cycle detected in the dependency graph.
